![](https://i.postimg.cc/xdfWcPmC/lv7q5yqe.png)

**Dataset Source:** *The dataset for "Corporate Compensation Insights" was collected from a leading multinational corporation known for its diverse workforce and dynamic business units. The dataset includes anonymized information on employee profiles, annual salaries, bonuses, job titles, business units, exit dates, gender distribution, and years of experience.*


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import missingno as msno
import plotly.express as px
import squarify
plt.style.use('fivethirtyeight')

In [ ]:
# imporing the dataset
Employee = pd.read_csv('/kaggle/input/corporate-compensation-insights/Employee Sample Data.csv',encoding='latin1')
df = Employee.copy()

In [ ]:
# information about data
df.info()

In [ ]:
# Checking Nan Value
msno.bar(df)

In [ ]:
# missing Value
for i in df.columns:
  nan_col = df[i].isna().sum() / len(df) * 100
  if nan_col > 0:
    print("{} nan_col {}%" .format(i,round(nan_col,2)))

**It's evident that the "Exit Date" column contains a substantial 91.5% of missing values. Since these cannot be imputed, we must opt to drop this column to facilitate further analysis.** 📊🚫🗑️

<div style="border: 60px solid #F05454; padding: 15px;">
    <span style="font-size: 35px; color: green;">$$Data Cleaning$$</span>
</div>

In [ ]:
# droping Exit Date
exit_date_drop = df.drop(['Exit Date'],axis=1, inplace=True)

In [ ]:
# converting object to datetime dtype
df['Hire Date'] = pd.to_datetime(df['Hire Date'])

In [ ]:
# Remove currency symbols and commas
df['Annual Salary'] = df['Annual Salary'].str.replace('[\$,€¥]', '', regex=True)
df['Annual Salary'] = df['Annual Salary'].str.replace(',', '', regex=True)

# Convert the cleaned 'CurrencyStrings' column to a numeric data type
df['Annual Salary'] = pd.to_numeric(df['Annual Salary'], errors='coerce')

In [ ]:
# looking into Annual Salary column after removing the dollor sign and changing dtypes object to integer
df['Annual Salary'].head()

In [ ]:
# Clean the 'Bonus %' column by removing the percentage symbol
df['Bonus %'] = df['Bonus %'].str.replace('%','',regex=True)

# Convert the cleaned 'Bonus %' column to a numeric data type (int)
df['Bonus %'] = df['Bonus %'].astype('int')

In [ ]:
df.info()

In [ ]:
styled_df = df.describe().style

# Set background color, text color, and border for the entire DataFrame
styled_df.set_properties(**{"background-color": "#7b5973", "color": "#e9c46a", "border": "1.5px solid black"})

# Modify the color and background color of the table headers (th)
styled_df.set_table_styles([
    {"selector": "th", "props": [("color", 'white'), ("background-color", "#0de601")]}
])

In [ ]:
# checking nunique values
for column in df.columns:
    print(f"{column}: Number of unique values {df[column].nunique()}")
    print("==========================================================")


In [ ]:
object_col = []
for column in df.columns:
    if df[column].dtype == object and len(df[column].unique()) <= 30:
        object_col.append(column)
        print(f"{column} : {df[column].unique()}")
        print(df[column].value_counts())
        print("***********************************************************")
#object_col.remove('Exit Date')
print('===================================================================')
print(f"Lenth of Categorical Columns:- {len(object_col)}")


<div style="border: 60px solid #F05454; padding: 15px;">
    <span style="font-size: 35px; color: green;">$$Data Visualization$$</span>
</div>

In [ ]:
# Assuming you have a DataFrame named 'df' and the 'Job Title' column
corpus = ' '.join(df['Job Title'])

# Customize WordCloud options
wordcloud = WordCloud(max_font_size=150, background_color='black', colormap='viridis',
                      width=1000, height=600, contour_color='black',
                      contour_width=2, font_path=None, min_font_size=12).generate(corpus)

# Create a plot
plt.figure(figsize=(15, 10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")

# Set the title
plt.title('Job Title Word Cloud', fontsize=30)

# Show the WordCloud
plt.show()


### In the word cloud, big words like **"Director,"** **"Sr Manager,"** **"Vice President,"** and **"Sr Analyst"** are the most common job titles. This helps us quickly understand which job roles appear most often in the data. It's a visual snapshot for decision-making.

In [ ]:
# Checking the Outliers
plt.figure(figsize=(10,8))
sns.boxplot(y=df['Annual Salary'],x=df['Business Unit'])
plt.show()

In [ ]:
# First of all, we are going to take only the subset of data for our purpose. (To keep things simple)
df_bar = df[['Business Unit','Annual Salary']]

# Grouping the DataFrame based on Business Unit and taking mean of Annual Salary across each of them.
# The reason for sorting is to show the larger values at the beginning (Best Practice)
df_bar = df_bar.groupby('Business Unit').mean().sort_values(by='Annual Salary', ascending=False)

plt.figure(figsize=[15,8]) # Setting the figure size

# Visualizing using Bar Chart
plt.suptitle("Average Annual Salary Across Different Business Unit", size=20)

# Plotting the Horizontal BarChart (Use this if there are many unique values for a Categorical Feature)
plt.barh(y=df_bar.index, width='Annual Salary', color=['#F05454','#30475E','#222831','#DDDDDD'], data=df_bar)
plt.gca().invert_yaxis() # Inverting the Y Axis

plt.show()

**The Corporate Business Unit clearly stands out with significantly higher salary levels compared to the other Business Units.** 💼💰🚀

In [ ]:
# First of all, we are going to take only the subset of data for our purpose. (To keep things simple)
df_line = df[['Hire Date','Annual Salary','Bonus %']].sort_values('Hire Date') # Chronological Ordering
df_line['Hire Date'] = pd.to_datetime(df_line['Hire Date']) # Converting into DateTime
df_line = df_line.groupby('Hire Date').mean() # Groupby to get the average Annual Salary and Bonus % on each year

# Visualizing the Line Chart
plt.figure(figsize=[15,8])
plt.plot(df_line.index, 'Annual Salary', data=df_line, color='#F05454') # Avg Annual Salary over Time
plt.plot(df_line.index, 'Bonus %', data=df_line, color='#30475E') # Avg Bonus % over Time
plt.title("Average Annual Salary and Bonus % over Time Period(1992-2020)", size=20, pad=20)

plt.show()

**These trends in increasing salaries and bonuses over time can help organizations make more informed decisions about compensation and experience-based incentives for their employees.** 📈💰✨💡

In [ ]:
# First of all, we are going to take only the subset of data for our purpose. (To keep things simple)
df_dist = df[['Annual Salary','Age']]

################################################################
# Checking The Distribution (It is okay if you do not understand this)
'''I am doing this so that we can see the visuals better. Do not
do this randomly'''

df_dist = df_dist[df_dist['Annual Salary'] <= 258498]
df_dist = df_dist[df_dist['Annual Salary'] <= 258498]
df_dist = df_dist[df_dist['Age'] >= 25]

################################################################

# Visualizing the KDE PLOT
plt.figure(figsize=[18,8])

plt.suptitle("Checking Distribution for Annual Salary By Age", size=20)

# KDEPlot for Annual Salary
plt.subplot(2,2,1)
sns.kdeplot(x=df_dist['Annual Salary'], color='#F05454')

# KDEPlot for Age Distribution
plt.subplot(2,2,2)
sns.kdeplot(x=df_dist['Age'], color='#30475E')

plt.show()

**In these two KDE plots, we can see that a significant number of people receive annual salaries ranging from 50K to 100K dollars. The second graph illustrates that individuals in the age group of 30 to 50 typically earn salaries within that same 50K to 100K dollar range.** 📊💰👥📈

In [ ]:
# Again taking a subset of data (To keep things simple)
df_scatter = df[['Annual Salary','Bonus %','Business Unit']]

# Visualizing the ScatterPlot
plt.figure(figsize=[15,8])
# Bonus % in the Y axis, and Annual Salary in the X. Hue will classify the dots according to Business Unit.
# The size of the dots are according to the volumen of "Annual Salary".
sns.scatterplot(x=df_scatter['Annual Salary'], y=df_scatter['Bonus %'], hue=df_scatter['Business Unit'], palette=['#F05454','#30475E','#222831','green'], size=df_scatter["Annual Salary"], sizes=(100,1000), legend='auto')
plt.title("Annual Salary vs Bonus % Across Different Business Unit", size=20, pad=20)
plt.show()

**In this scatter plot, we can clearly see that the business unit specializing in specialty products receives higher salaries and bonuses compared to others.** 💼💰🚀

### 📌Using The Stacked BarChart to show composition for mutiple features is sometimes called Bi-Variate or Multi-Variate Analysis.

In [ ]:
# Taking a subset of data (To keep things simple)
df_stackb = df[['Department', 'Annual Salary', 'Bonus %']]
# taking a groupby on Department and then Bonus %...
df_stackb = df_stackb.groupby(['Department']).sum().reset_index()

# Define colors for the stacked bar chart
bar_colors = ['#F05454', '#30475E']

# Visualizing the Stacked BarChart
plt.figure(figsize=[15, 15])
plt.subplot(3, 1, 1)
plt.bar(x=df_stackb['Department'], height=df_stackb['Annual Salary'], color=bar_colors[0], label='Annual Salary')
plt.bar(x=df_stackb['Department'], height=df_stackb['Bonus %'], bottom=df_stackb['Annual Salary'], color=bar_colors[1], label='Bonus %')
plt.title("Annual Salary & Bonus % Breakdown by Department", size=20, pad=20)
plt.legend()

# Taking a subset of data (To keep things simple)
df_pie = df[['Business Unit', 'Annual Salary', 'Bonus %']]
# taking a groupby on Business Unit and then Gender...
df_pie = df_pie.groupby(['Business Unit']).sum().reset_index()
df_gender = df[['Gender']]
df_gender = df_gender.groupby(['Gender']).size().reset_index()  # Count the occurrences of each gender

# Define colors for the pie charts
gender_colors = ['#F05454', '#30475E']
city_colors = ['#F05454', '#30475E', '#222831', '#DDDDDD']

# Visualizing the Pie Chart (Bonus %)
plt.subplot(3, 2, 3)
plt.pie(df_gender[0], labels=df_gender['Gender'], autopct='%1.1f%%',
       shadow=True, startangle=90, colors=gender_colors)
plt.title("Gender-wise Bonus Percentage")

# Visualizing the Pie Chart (Annual Salary)
plt.subplot(3, 2, 4)
plt.pie(df_pie['Annual Salary'], labels=df_pie['Business Unit'],hatch=['**O', 'oO', 'O.O', '.||.'], shadow=True, startangle=180, colors=city_colors)
plt.title("Annual Salary Distribution by Business Unit")

plt.show()


### **In the bar graph showing how annual salaries and bonus percentages are divided by department, it's clear that the IT department gets the highest salaries and bonuses.**

- **The first pie chart, which looks at the percentage of bonuses based on gender, tells us that females receive larger bonuses compared to males.**

- **Lastly, the second pie chart, displaying the distribution of annual salaries in different business units, gives us an idea of how salaries are spread across these units.**

In [ ]:
!pip install squarify

In [ ]:
# Taking a subset of data (To keep things simple)
df_tree = df[['Business Unit','Annual Salary']]
# taking a groupby on Business Unit and then Annual Salary...
df_tree = df_tree.groupby(['Business Unit']).sum().reset_index()

# Visualizing the TreeMap (Annual Salary)
plt.figure(figsize=[10,8])

squarify.plot(sizes=df_tree['Annual Salary'], label=df_tree['Business Unit'],
              color=['#F05454','#30475E','#222831'], alpha=0.7)
plt.title("Annual Salary Across Different Business Unit", size=20, pad=20)
plt.axis('off')

plt.show()

**In this treemap, it's evident that there is a strong connection between annual salaries in the Corporate and Specialty Products sectors. Employees in these sectors generally receive higher salaries.**

In [ ]:
# taking a groupby on Age and then Gender with their Bonus %...
age_att = df.groupby(['Age', 'Gender']).apply(lambda x: x['Bonus %'].count()).reset_index(name='Counts')
px.line(age_att, x='Age', y='Counts', color='Gender', title='Age by Counts of Gender in an Organization')

**This graph is like a rollercoaster ride through age groups at work. The most exciting part is between ages 45 and 55, where both men and women do really well. But, women get even better bonuses in this comparison. This information can help in making informed decisions. 📊💫💼**


## **Informed Decision-Making Insights:🚀**

- **Data Cleaning:** The dataset has been meticulously cleaned, setting the stage for robust analysis.

- **Job Titles Snapshot:** A word cloud highlights top job titles, like "Director," "Sr Manager," "Vice President," and "Sr Analyst." It offers a quick understanding of the most common roles.

- **Salary Correlation:** A treemap reveals a strong connection between Corporate and Specialty Products sectors, where higher salaries are the norm.

- **Salaries & Bonuses Trend:** Time-related data displays an upward trend in salaries and bonuses, aiding strategic decisions on compensation structures and employee incentives.

- **Corporate Distinction:** The Corporate Business Unit stands out with significantly higher salaries, demanding focused attention.

- **Experience Pays Off:** Experience years positively impact salaries and bonuses, signifying the value of experience-based incentives.

- **Business Unit Insights:** The Corporate Business Unit leads in salary levels, prompting considerations for resource allocation.

- **Data Gaps Decision:** The "Exit Date" column contains 91.5% missing data, calling for a decision on whether to fill the gaps or remove the column for further analysis.

- **Gender Pay Equity:** Notably, women receive higher bonuses than men, sparking discussions on pay equity and HR policies.

- **Salary Range Reference:** Most employees fall within the $50,000 to $100,000 (50K to 100K) salary range, serving as a benchmark for competitive compensation.

- **Unit-Specific Needs:** The Specialty Product Business Unit is associated with higher compensation, shedding light on unit-specific needs.

- **Prime Earning Years:** The age range of 30 to 50 marks the prime earning years, guiding HR and development strategies.


## Conclusion:

### "Corporate Compensation Insights" allows us to make data-informed decisions to foster a more equitable, competitive, and rewarding work environment. We need to explore gender equity, optimize salary budget allocation, and leverage our workforce's expertise to drive the company's success. 🚀"

<div style="border: 2px solid green; padding: 10px;">
    <span style="color: green; font-size: 30px;">If you enjoyed this Notebook, please do upvote.👍</span>
    <br>
    <span style="color: blue; font-size: 30px;">If you have any questions, feel free to comment! 💬</span>
    <br>
    <span style="color: purple; font-size: 30px;">Best Wishes! 🌟</span>
</div>
